In [1]:
import pandas as pd


In [2]:
data = pd.read_csv("/content/drive/MyDrive/Model Deployment/LoanApp V_2/Deploying Machine Learning model using Streamlit/loan_data.csv")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data.head()

,Gender,Married,ApplicantIncome,LoanAmount,Loan_Status
0,Male,Yes,4583,128000.0,N
1,Male,Yes,3000,66000.0,Y
2,Male,Yes,2583,120000.0,Y
3,Male,No,6000,141000.0,Y
4,Male,Yes,5417,267000.0,Y


In [5]:
data.Gender = data.Gender.map({"Male" : 1 , "Female" : 0})

In [6]:
data.Married = data.Married.map({"Yes" : 1 , "No" : 0})

In [7]:
data.Loan_Status = data.Loan_Status.map({"Y" : 1 , "N":0})

In [8]:
aIncome_max = data.ApplicantIncome.max()
aIncome_min = data.ApplicantIncome.min()
loanAmount_max = data.LoanAmount.max()
loanAmount_min = data.LoanAmount.min()

In [9]:
# data.ApplicantIncome = (data.ApplicantIncome - aIncome_min)/(aIncome_max)
# data.LoanAmount = (data.LoanAmount - loanAmount_min)/(loanAmount_max)

In [10]:
X = data.drop("Loan_Status", axis=1)
y = data["Loan_Status"]

In [11]:
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.initializers import random_normal

In [12]:
from sklearn.model_selection import train_test_split as tts
train_x,val_x, train_y, val_y = tts(X,y,test_size= 0.10,stratify=y)

In [13]:
train_x.shape, val_x.shape

((519, 4), (58, 4))

In [14]:
model = Sequential()
model.add(InputLayer(input_shape=(train_x.shape[1])))
model.add(Dense(units=30, activation="relu", kernel_initializer="random_normal"))
model.add(Dense(units = 10 , activation="relu", kernel_initializer="random_normal"))
model.add(Dense(units=1 , activation="sigmoid"))

In [15]:
model.compile(loss = "binary_crossentropy" , optimizer="Adam" , metrics="accuracy")

In [16]:
from keras.callbacks import ModelCheckpoint
filepath = 'checkpoint.hdf5'
model_checkpoint_callback = ModelCheckpoint(filepath,monitor="val_accuracy" , verbose=1,save_best_only=True,)

In [17]:
model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs = 100, batch_size = 100, callbacks=model_checkpoint_callback)

Epoch 1/100
1/6 [====>.........................] - ETA: 12s - loss: 1.9799 - accuracy: 0.7200
Epoch 1: val_accuracy improved from -inf to 0.68966, saving model to checkpoint.hdf5
6/6 [==============================] - 3s 46ms/step - loss: 38.0324 - accuracy: 0.5588 - val_loss: 32.2154 - val_accuracy: 0.6897
Epoch 2/100
1/6 [====>.........................] - ETA: 0s - loss: 25.9577 - accuracy: 0.7100
Epoch 2: val_accuracy did not improve from 0.68966
6/6 [==============================] - 0s 9ms/step - loss: 19.2473 - accuracy: 0.5299 - val_loss: 12.2990 - val_accuracy: 0.6897
Epoch 3/100
1/6 [====>.........................] - ETA: 0s - loss: 8.7163 - accuracy: 0.7200
Epoch 3: val_accuracy did not improve from 0.68966
6/6 [==============================] - 0s 8ms/step - loss: 19.8722 - accuracy: 0.6956 - val_loss: 5.9015 - val_accuracy: 0.3103
Epoch 4/100
1/6 [====>.........................] - ETA: 0s - loss: 5.0132 - accuracy: 0.3900
Epoch 4: val_accuracy did not improve from 0.68966
6

In [18]:
model.load_weights(filepath="checkpoint.hdf5")

In [19]:
import pickle

In [20]:
model_store = open("model.pkl", "wb")
pickle.dump(model , model_store)
model_store.close()

In [21]:
# !pip install streamlit
# !pip install pyngrok
!ngrok authtoken 2OUvp7wQtqaCWchjOnajYgkug2c_83YMdMGU7WHikiS69um32

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


17/17 [==============================] - 0s 3ms/step


array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [

In [31]:
%%writefile loanapp_v2.py
import pandas as pd
import streamlit as st
import pickle
import numpy as np
pickle_in = open('model.pkl', 'rb') 
model = pickle.load(pickle_in)
def main():
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Check your Loan Eligibility</h1> 
    </div> 
    """
    st.markdown(html_temp, unsafe_allow_html = True) 
    
    Gender = st.selectbox('Gender',("Male","Female"))
    Married = st.selectbox('Marital Status',("Unmarried","Married")) 
    ApplicantIncome = st.number_input("Monthly Income in Rupees") 
    LoanAmount = st.number_input("Loan Amount in Rupees")
    
    
    if st.button("Check"): 
      result = prediction(Gender, Married, ApplicantIncome, LoanAmount) 
      st.success('Your loan is {}'.format(result))
    
def prediction(Gender, Married, ApplicantIncome, LoanAmount):
    
    if Gender == "Male":
        Gender = 1
    else:
        Gender = 0

    if Married == "Married":
        Married = 1
    else:
        Married = 0
    pred = (model.predict([[Gender, Married, ApplicantIncome, LoanAmount]]) > 0.5)
    if pred[0][0] == False:
      return "Not Approved"
    else:
      return "Approved"
    
if __name__=='__main__': 
    main()
        
        
        

Overwriting loanapp_v2.py


In [32]:
!streamlit run loanapp_v2.py &>/dev/null&

In [57]:
from pyngrok import ngrok